In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from scipy import stats
from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d

import matplotlib.pyplot as plt

from fimpylab.core.lightsheet_experiment import LightsheetExperiment

import json
from pathlib import Path

from vision_and_navigation.imaging.general import corr2_coeff
from vision_and_navigation.imaging.visual_motion import make_sensory_regressors

# Set fish path

In [ ]:
master = Path(r"Z:\Hagar and Ot\E0040\v10\LS")

fig_path = master # master # None

fish_list = list(master.glob("*_f*"))

# Load morphed coordinates

In [ ]:
morphed_coords = {}
in_brain_arr = {}

for fish in fish_list:
    print(fish)
    
    #Load morphed coords
    morphed_coords[fish.name] = fl.load(fish / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'mov_coords_transformed.h5')
    
    #Index to keep track of ROIs inside brain
    suite2p_brain = fl.load(fish / "data_from_suite2p_cells_brain.h5")
    
    in_brain_arr[fish.name] = np.full(morphed_coords[fish.name].shape[0], False)
    in_brain_arr[fish.name][suite2p_brain['coords_idx']] = True

    
print('Done')

In [ ]:
#And pool in a single array
coords_pooled = np.concatenate([morphed_coords[fish.name] for fish in fish_list], 0)

# Sensory regressors

In [ ]:
titles = ['right', 'backward right', 'backward', 'backward left', 'left', 'forward left', 'forward', 'forward right', ]

In [ ]:
for plot_dir in range(8):
    try:
        reg_corrcoefs_pooled = fl.load(master / 'reg_corrcoefs_pooled_dir{}.h5'.format(plot_dir))

    except OSError:
        reg_corrcoefs = []

        for path in fish_list:
            traces = fl.load(path / "filtered_traces.h5", "/detr")
            sensory_regressors = fl.load(path / "sensory_regressors.h5", "/regressors")

            current_dir = np.asarray(sensory_regressors.iloc[:, plot_dir])        
            reg_corrcoefs.append(corr2_coeff(traces.T, current_dir.reshape(1, -1)).ravel())

        reg_corrcoefs_pooled = np.concatenate(reg_corrcoefs)
        fl.save(master / 'reg_corrcoefs_pooled_dir{}.h5'.format(plot_dir), reg_corrcoefs_pooled)

In [ ]:
plot_dir = 7
reg_corrcoefs_pooled = fl.load(master / 'reg_corrcoefs_pooled_dir{}.h5'.format(plot_dir))

In [ ]:
in_brain_arr_pooled = np.concatenate([in_brain_arr[fish.name] for fish in fish_list])

In [ ]:
#Filter ROIs and normalize vector amplitude
coords_ib_pooled = coords_pooled[in_brain_arr_pooled]
reg_corrcoefs_ib_pooled = reg_corrcoefs_pooled[in_brain_arr_pooled]
mp_ind_regressor = np.argsort(np.abs(reg_corrcoefs_ib_pooled))

In [ ]:
dot_s = 0.1

scale_bar_len = 100
scale_bar_xpos = 10
scale_bar_ypos1 = 650
fs = 8

In [ ]:
#Import reference anatomy [from first fish]
ref_anatomy = fl.load(fish_list[0] / 'registration' / 'to_h2b_baier_ref' / 'antspy' / 'ref_mapped.h5')

In [ ]:
x_crop = [30, 580]
y_crop = [40, 740]
z_crop = [0, ref_anatomy.shape[2]]

ref_anatomy_cropped = ref_anatomy[x_crop[0]:x_crop[1], y_crop[0]:y_crop[1], z_crop[0]:z_crop[1]]

In [ ]:
#Plot
y_main_rat = ref_anatomy_cropped.shape[1]/ref_anatomy_cropped.shape[0]
x_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[0]
y_rat = ref_anatomy_cropped.shape[2]/ref_anatomy_cropped.shape[1]*y_main_rat

fig = plt.figure(figsize=(3.5,4))
gs = fig.add_gridspec(2, 2, width_ratios=[1, x_rat], height_ratios=[y_rat, y_main_rat])

ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[0, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[1, 1], sharey=ax1)

mock_anatomy = np.zeros_like(ref_anatomy_cropped)
ax1.imshow(mock_anatomy.mean(2).T, cmap='gray_r')
ax2.imshow(mock_anatomy.mean(1).T, cmap='gray_r')
ax3.imshow(mock_anatomy.mean(0), cmap='gray_r')

ax1.scatter(coords_ib_pooled[mp_ind_regressor,0]-x_crop[0], coords_ib_pooled[mp_ind_regressor,1]-y_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
ax2.scatter(coords_ib_pooled[mp_ind_regressor,0]-x_crop[0], coords_ib_pooled[mp_ind_regressor,2]-z_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
ax3.scatter(coords_ib_pooled[mp_ind_regressor,2]-z_crop[0], coords_ib_pooled[mp_ind_regressor,1]-y_crop[0], c=reg_corrcoefs_ib_pooled[mp_ind_regressor], s=dot_s, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

#Scale bars
ax1.plot((scale_bar_xpos, scale_bar_xpos+scale_bar_len), (scale_bar_ypos1, scale_bar_ypos1), c='black')
# ax1.text(scale_bar_xpos+(scale_bar_len/2), scale_bar_ypos1+10, r'{}$\mu$m'.format(scale_bar_len), va='top', ha='center', fontsize=fs)

for ax in [ax1, ax2, ax3]:
    ax.set_aspect('equal')
    ax.axis('off')
    
ax1.set_xlim([0, x_crop[1]-x_crop[0]])
ax1.set_ylim([0, y_crop[1]-y_crop[0]])

ax2.set_xlim([0, x_crop[1]-x_crop[0]])
ax2.set_ylim([0, z_crop[1]-z_crop[0]])

ax3.set_xlim([0, z_crop[1]-z_crop[0]])
ax3.set_ylim([0, y_crop[1]-y_crop[0]])

ax1.invert_yaxis()

plt.subplots_adjust(hspace=0, wspace=0, left=0.05, right=0.95, top=0.95, bottom=0.05)


In [ ]:
plt.subplots_adjust(hspace=0, wspace=0.0, left=0.05, right=0.95, top=0.95, bottom=0.05)


In [ ]:
if fig_path is not None:
    fig.savefig(fig_path / 'pooled_regressor_corrcoefs_{}.png'.format(plot_dir), dpi=300)